In [1]:
#packages:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
import pickle

In [2]:
from transforms3d import quaternions

In [3]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [4]:
import torch

In [5]:
import plane_calib_pytorch as pcp
import convert_dataset as cdt
import engine

Setting up a new session...


In [6]:
from leddar_utils import linalg

In [7]:
import das.imu as DI

In [7]:
with open('/nas/pixset/exportedDataset/20200519_201946_rec_dataset_calib_imu_brick03_exported/extrinsics/sbgekinox_bcc-ouster64_bfc.pkl', 'rb') as k:
    init_matrix = pickle.load(k)

In [8]:
init_matrix

array([[ 0.99965439, -0.01256657, -0.0230906 , -3.91731072],
       [ 0.01240368,  0.99989726, -0.0071842 , -0.03624008],
       [ 0.02317851,  0.00689531,  0.99970756, -0.31468444],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [10]:
DI.print_transf_result(init_matrix)

translations :[-3.91731072 -0.03624008 -0.31468444], norm:3.930097058768463
Ax Angle vector:[ 0.25867401 -0.85007324  0.45876274], angle (deg):1.559484577813044
Euler Angle (deg):[ 0.39518149 -1.32814976  0.7108878 ]


In [12]:
result_tr_1 = np.load('/home/olivier/workspace/das/das/calib/examples/extrinsic_sbgekinox_ouster64/ckps7b/results.npy', allow_pickle=True).item()

In [13]:
result_tr_1.keys()

dict_keys(['loss', 'calib', 'bias'])

In [14]:
M_trs = result_tr_1['calib']
len(M_trs)

45

In [15]:
M_trs[-1]

array([[ 0.99958146,  0.00745099,  0.02795299,  3.8970003 ],
       [-0.0075772 ,  0.99996156,  0.00441208, -0.01547982],
       [-0.02791904, -0.00462204,  0.9995995 ,  0.21045506],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32)

In [59]:
M = linalg.tf_inv(M_trs[60])
#test:
M[2,3]=-10.40
M

array([[ 9.9958199e-01, -5.8814264e-03, -2.8304942e-02, -3.8763850e+00],
       [ 5.7504629e-03,  9.9997240e-01, -4.7058542e-03, -2.4932658e-02],
       [ 2.8331839e-02,  4.5411135e-03,  9.9958825e-01, -1.0400000e+01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32)

In [53]:
DI.print_transf_result(linalg.tf_inv(M_trs[60]))

translations :[-3.87638497 -0.02493266 -0.40184802], norm:3.897237987361658
Ax Angle vector:[ 0.15792294 -0.96726267  0.19865363], angle (deg):1.6776773523226292
Euler Angle (deg):[ 0.26029202 -1.62351202  0.3296114 ]


In [60]:
with open('sbgekinox_bcc-ouster64_bfc.pkl', 'wb') as k:
    pickle.dump(M, k)

In [9]:
plane_calib = pcp.PlaneCalib(init_matrix, init_bias = np.array([0.12]), use_bias=True)

In [10]:
plane_calib.get_transformation_matrix()

tensor([[ 0.9997, -0.0126, -0.0231, -3.9173],
        [ 0.0124,  0.9999, -0.0072, -0.0362],
        [ 0.0232,  0.0069,  0.9997, -0.3147],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], grad_fn=<CopySlices>)

In [10]:
for params in plane_calib.parameters():
    print(params)

Parameter containing:
tensor([ 0.0070, -0.0231,  0.0125], requires_grad=True)
Parameter containing:
tensor([-3.9173, -0.0362, -0.3147], requires_grad=True)
Parameter containing:
tensor([0.1200], requires_grad=True)


In [11]:
#Validation--set a plane:
xyz = np.array([
    [-1.04194694, -1.17965867,  1.09517722],
    [-0.39947906, -1.37104542,  1.36019265],
    [-1.0634807 , -1.35020616,  0.46773962],
    [-0.48640524, -1.64476106,  0.2726187 ],
    [-0.05720509, -1.6791781 ,  0.76964551],
    [-1.27522669, -1.10240358,  0.33761405],
    [-0.61274031, -1.52709874, -0.09945502],
    [-1.402693  , -0.86807757,  0.88866091],
    [-0.72520241, -0.86800727,  1.69729388]])

In [ ]:
#method valid:
#P: ax+by+cz +d =0
reg = LinearRegression()
Y = xyz.copy()
x_fit, y_fit = Y[:,1:], Y[:,0:1] #here I try to fit: x = w0 y + w1 z + w2
reg.fit(x_fit,y_fit)
P = (-1, reg.coef_[0][0], reg.coef_[0][1], reg.intercept_[0])

In [ ]:
n_hat = np.array([P[0], P[1], P[2]])
n_hat /= np.linalg.norm(n_hat)
n_hat

In [ ]:
n,d =plane_calib.fit_plane(torch.from_numpy(xyz).float())

In [ ]:
dists = plane_calib.distance_points_to_plane(torch.from_numpy(xyz).float(),n,d)

In [ ]:
dists

In [17]:
x = torch.from_numpy(xyz).float()

In [18]:
x = torch.stack([x+0.5,x, x+0.9,x], dim=0)

In [19]:
x.size()

torch.Size([4, 9, 3])

In [10]:
u,s,v = torch.svd(x)

In [11]:
u.size()

torch.Size([2, 9, 3])

In [12]:
s

tensor([[5.0764, 1.6666, 1.4023],
        [5.0764, 1.6666, 1.4023]])

In [13]:
v.size()

torch.Size([2, 3, 3])

In [15]:
n,d=plane_calib.fit_plane(x)

In [16]:
n.size(), n

(torch.Size([2, 3]), tensor([[-0.4690, -0.8409,  0.2702],
         [-0.4690, -0.8409,  0.2702]]))

In [17]:
d.size(), d

(torch.Size([2]), tensor([-1.6548, -1.6548]))

In [18]:
r = torch.randn(3,3)
x.size()

torch.Size([2, 9, 3])

In [30]:
c =torch.matmul(r, x.transpose(1,2))

In [31]:
c.size()

torch.Size([2, 3, 9])

In [37]:
r.squeeze(0).expand(2,-1,-1)

tensor([[[-0.9040, -1.1916,  1.0402],
         [ 2.0101, -0.1697,  0.3011],
         [-0.8189,  1.0308, -0.3259]],

        [[-0.9040, -1.1916,  1.0402],
         [ 2.0101, -0.1697,  0.3011],
         [-0.8189,  1.0308, -0.3259]]])

In [32]:
d = torch.bmm(r.squeeze(0).expand(2,-1,-1), x.transpose(1,2))

In [33]:
d.size()

torch.Size([2, 3, 9])

In [34]:
c == d

tensor([[[ True, False,  True,  True,  True,  True, False,  True, False],
         [ True,  True,  True, False,  True,  True,  True,  True,  True],
         [ True,  True, False,  True,  True,  True,  True, False,  True]],

        [[False, False,  True,  True,  True,  True,  True,  True,  True],
         [ True, False,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True, False,  True]]])

In [38]:
c-d

tensor([[[ 0.0000e+00,  2.3842e-07,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00, -1.1921e-07,  0.0000e+00, -2.3842e-07],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  2.9802e-08,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -5.9605e-08,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00, -7.4506e-09,  0.0000e+00]],

        [[ 2.3842e-07,  2.3842e-07,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00, -1.4901e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00, -7.4506e-09,  0.0000e+00]]])

In [20]:
l = plane_calib(x)

In [21]:
l.size()

torch.Size([4, 9])

In [24]:
l[1]

tensor([0.1280, 0.0559, 0.1117, 0.0320, 0.0089, 0.0396, 0.1165, 0.0299, 0.1327],
       grad_fn=<SelectBackward>)

In [25]:
l[3]

tensor([0.1280, 0.0559, 0.1117, 0.0320, 0.0089, 0.0396, 0.1165, 0.0299, 0.1327],
       grad_fn=<SelectBackward>)

In [26]:
l[0]

tensor([0.1263, 0.0571, 0.1132, 0.0313, 0.0133, 0.0398, 0.1114, 0.0353, 0.1282],
       grad_fn=<SelectBackward>)

In [13]:
optimizer = torch.optim.SGD(plane_calib.parameters(), lr=.1, momentum=0.9)

In [14]:
N_ITERATIONS = 250
for i in range(N_ITERATIONS):
    optimizer.zero_grad()
    l = plane_calib(x)
    l.backward()
    optimizer.step()
    print(l)
    

/pytorch/torch/csrc/utils/python_arg_parser.cpp:698: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, Number alpha)


tensor(0.0728, grad_fn=<MeanBackward0>)
tensor(0.0727, grad_fn=<MeanBackward0>)
tensor(0.0725, grad_fn=<MeanBackward0>)
tensor(0.0722, grad_fn=<MeanBackward0>)
tensor(0.0719, grad_fn=<MeanBackward0>)
tensor(0.0714, grad_fn=<MeanBackward0>)
tensor(0.0709, grad_fn=<MeanBackward0>)
tensor(0.0704, grad_fn=<MeanBackward0>)
tensor(0.0698, grad_fn=<MeanBackward0>)
tensor(0.0692, grad_fn=<MeanBackward0>)
tensor(0.0685, grad_fn=<MeanBackward0>)
tensor(0.0678, grad_fn=<MeanBackward0>)
tensor(0.0670, grad_fn=<MeanBackward0>)
tensor(0.0662, grad_fn=<MeanBackward0>)
tensor(0.0654, grad_fn=<MeanBackward0>)
tensor(0.0646, grad_fn=<MeanBackward0>)
tensor(0.0637, grad_fn=<MeanBackward0>)
tensor(0.0629, grad_fn=<MeanBackward0>)
tensor(0.0620, grad_fn=<MeanBackward0>)
tensor(0.0611, grad_fn=<MeanBackward0>)
tensor(0.0602, grad_fn=<MeanBackward0>)
tensor(0.0592, grad_fn=<MeanBackward0>)
tensor(0.0583, grad_fn=<MeanBackward0>)
tensor(0.0573, grad_fn=<MeanBackward0>)
tensor(0.0564, grad_fn=<MeanBackward0>)


tensor(0.0153, grad_fn=<MeanBackward0>)


In [15]:
plane_calib.get_transformation_matrix()

tensor([[ 0.9997, -0.0126, -0.0231, -3.9173],
        [ 0.0124,  0.9999, -0.0072, -0.0362],
        [ 0.0232,  0.0069,  0.9997, -0.3147],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], grad_fn=<CopySlices>)

In [16]:
plane_calib.init_matrix-plane_calib.get_transformation_matrix().detach().numpy()

array([[-1.93157809e-08, -2.83296050e-06,  1.11668693e-06,
        -1.69490466e-09],
       [ 2.79036223e-06, -3.85046187e-08, -2.56533012e-06,
         7.10964845e-07],
       [-1.06912883e-06,  2.53017400e-06,  4.29067488e-08,
         7.00548530e-09],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

In [17]:
plane_calib.bias

Parameter containing:
tensor([-1.8441], requires_grad=True)

In [18]:
plane_calib.init_bias

array([0.12])

In [20]:
quaternions.mat2quat(np.eye(3))

array([1., 0., 0., 0.])

In [ ]:
#using the dataset prepared by Pierre
# (testing)
#

In [11]:
dataset = '/nas/pixset/exportedDataset/20200519_201503_rec_dataset_calib_imu_brick02_exported'
plane_cfg = 'planes_ouster_20200519_201503.yml'


exportation_dataset = cdt.DatasetExportation(dataset, plane_cfg, 'ouster64_bfc')
exportation_dataset.load_from_pickle_file()

 22%|██▏       | 2/9 [00:00<00:01,  4.85it/s]

Pixell Log: the cache is set to use calibration angles.
Pixell Log: time base delays are converted to lcas referential.


100%|██████████| 9/9 [00:01<00:00,  6.90it/s]
/home/olivier/workspace/das/das/calib/examples/extrinsic_sbgekinox_ouster64/convert_dataset.py:198: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.scene = yaml.load(f)


nb frame nav:11693
nb frame euler:11693
position offset: [3.20495141e+05 5.18384259e+06 1.73132630e+01]
nb frame ouster64_bfc:2342


In [12]:
len(exportation_dataset)

9

In [30]:
init_matrix_inv = linalg.tf_inv(init_matrix)

In [45]:
pts, tr = exportation_dataset.get_points(2, init_matrix)
pts.shape, tr.shape

((3, 9000), (9000, 4, 4))

In [46]:
pts_hom = np.vstack((pts, np.ones(pts.shape[1])))
pts_hom.shape

(4, 9000)

In [47]:
pts_world = np.zeros_like(pts_hom)
for i in range(pts.shape[1]):
    pts_world[:,i:i+1] = tr[i]@init_matrix@pts_hom[:,i:i+1]

In [48]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pts_world[0], pts_world[1], pts_world[2])
plt.show()

In [11]:
plane_calib_dataset = cdt.PlaneCalibDataset(calibration_matrix=init_matrix)

 22%|██▏       | 2/9 [00:00<00:01,  4.57it/s]

Pixell Log: the cache is set to use calibration angles.
Pixell Log: time base delays are converted to lcas referential.


100%|██████████| 9/9 [00:01<00:00,  6.57it/s]
/home/olivier/workspace/das/das/calib/examples/extrinsic_sbgekinox_ouster64/convert_dataset.py:199: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.scene = yaml.load(f)


nb frame nav:11693
nb frame euler:11693
position offset: [3.20495141e+05 5.18384259e+06 1.73132630e+01]
nb frame ouster64_bfc:2342


In [12]:
len(plane_calib_dataset)

9

In [13]:
plane_calib_dataset.calibration_matrix

array([[ 0.99965439, -0.01256657, -0.0230906 , -3.91731072],
       [ 0.01240368,  0.99989726, -0.0071842 , -0.03624008],
       [ 0.02317851,  0.00689531,  0.99970756, -0.31468444],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [14]:
dataloader = cdt.PlaneCalibDataloader(plane_calib_dataset, 
                                      num_max_points=500, 
                                      batch_size=3, 
                                      device=torch.device('cuda'), 
                                      shuffle=True)

In [15]:
plane_calib.to(torch.device('cuda'))

PlaneCalib()

In [16]:
for pts,tr in dataloader:
    print(pts.size(), tr.size())
    l = plane_calib(pts, tr)

torch.Size([9, 500, 3]) torch.Size([9, 500, 4, 4])


In [17]:
l

tensor(1.9051, device='cuda:0', grad_fn=<SumBackward0>)

In [18]:
l.size()

torch.Size([9, 500])

In [7]:
with open('/nas/pixset/exportedDataset/20200519_203346_rec_dataset_brick_to_watt_exported/extrinsics/sbgekinox_bcc-ouster64_bfc.pkl', 'rb') as k:
    calib_matrix = pickle.load(k)

In [8]:
class ArgsConfig:
    #dataset:
    dataset = '/nas/pixset/exportedDataset/20200519_201503_rec_dataset_calib_imu_brick02_exported'
    planes_cfg='planes_ouster_20200519_201503.yml'
    sensor='ouster64_bfc'
    exported_package='exported_dataset.pkl'
    
    #loader:
    num_max_points=500
    batch_size=9
    device='cuda'
    shuffle=True
    
    #model:
    init_matrix = calib_matrix
    init_bias = np.array([0.02])
    use_bias = True
    
    
    lr=1e-4
    weight_decay=1e-4
    
    num_epochs=50
    
    
    

In [ ]:
args = ArgsConfig()
package = engine.main(args)

 22%|██▏       | 2/9 [00:00<00:01,  4.91it/s]

Pixell Log: the cache is set to use calibration angles.
Pixell Log: time base delays are converted to lcas referential.


100%|██████████| 9/9 [00:01<00:00,  6.61it/s]
/home/olivier/workspace/das/das/calib/examples/extrinsic_sbgekinox_ouster64/convert_dataset.py:199: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.scene = yaml.load(f)


nb frame nav:11693
nb frame euler:11693
position offset: [3.20495141e+05 5.18384259e+06 1.73132630e+01]
nb frame ouster64_bfc:2342
Start training


In [8]:
#validation via cloud=compare pour un dataset
from das.api import platform
import tqdm

In [18]:
#
val_ds = '/nas/pixset/exportedDataset/20200608_175725_rec_dataset_calib_imu_fast_brick04_exported'
pf_val = platform.Platform(val_ds)
#pts = []
#for mu in tqdm.tqdm(pf_val['ouster64_bfc_xyzit'][20:500]):
#    pts.append(mu.point_cloud(referential='world', ignore_orientation=False, undistort=True, reference_ts=-1, dtype=np.float64))
#pts = np.vstack(pts)
#print(pts.shape)

#np.savetxt("val_pts.xyz", pts, fmt='%.3f')

 10%|█         | 1/10 [00:03<00:34,  3.79s/it]

Pixell Log: the cache is set to use calibration angles.
Pixell Log: time base delays are converted to lcas referential.


100%|██████████| 10/10 [01:11<00:00,  7.14s/it]


In [16]:
pf_val

NameError: name 'pf_val' is not defined

In [38]:
z = pf_val['pixell_bfc']['ech'][10].raw['data']

In [39]:
len(z)

2304

In [40]:
x=pf_val['pixell_bfc']['ech'][10].point_cloud()

In [41]:
len(x)

502

In [42]:
pf_val['pixell_bfc']['ech'][10]._mask

array([ True, False, False, ...,  True, False, False])

In [37]:
pf_val['pixell_bfc']['ech'][10].timestamps

array([1591639048372928, 1591639048374334, 1591639048372928,
       1591639048374334, 1591639048372928, 1591639048374334,
       1591639048372928, 1591639048374334, 1591639048372928,
       1591639048374334, 1591639048372928, 1591639048374334,
       1591639048372928, 1591639048374334, 1591639048372928,
       1591639048374334, 1591639048372928, 1591639048374334,
       1591639048372928, 1591639048374334, 1591639048372928,
       1591639048374334, 1591639048372928, 1591639048374334,
       1591639048374334, 1591639048372928, 1591639048372928,
       1591639048374334, 1591639048355669, 1591639048357168,
       1591639048355669, 1591639048357168, 1591639048355669,
       1591639048357168, 1591639048355669, 1591639048357168,
       1591639048355669, 1591639048357168, 1591639048355669,
       1591639048357168, 1591639048355669, 1591639048357168,
       1591639048355669, 1591639048357168, 1591639048355669,
       1591639048357168, 1591639048355669, 1591639048357168,
       1591639048355669,

In [34]:
np.mod(z,2).astype(bool)

array([ True, False, False, ...,  True, False, False])

In [33]:
pf_val['pixell_bfc']['ech'][10].raw['data']

array([( 95, 21.751663, 19.5625, 1591639048372928, 129),
       ( 95,  0.      ,  0.    , 1591639048372928, 128),
       ( 95,  0.      ,  0.    , 1591639048372928, 128), ...,
       (672,  4.242218, 72.8125, 1591639048371508, 129),
       ( 95,  0.      ,  0.    , 1591639048372928, 128),
       ( 95,  0.      ,  0.    , 1591639048372928, 128)],
      dtype=[('indices', '<u4'), ('distances', '<f4'), ('amplitudes', '<f4'), ('timestamps', '<u8'), ('flags', '<u2')])

In [45]:
# initial : sbgekinox_bcc-pixell_bfc.pkl
with open('/nas/pixset/exportedDataset/20200608_180144_rec_dataset_calib_imu_fast_brick05_exported/extrinsics/sbgekinox_bcc-ouster64_bfc.pkl', 'rb') as k:
    tf_from_sbg_to_ous = pickle.load(k)

with open('/nas/pixset/exportedDataset/20200608_180144_rec_dataset_calib_imu_fast_brick05_exported/extrinsics/pixell_bfc-ouster64_bfc.pkl', 'rb') as k:
    tf_from_pix_to_ous = pickle.load(k)


tf_from_sbg_to_pix = tf_from_sbg_to_ous @ linalg.tf_inv(tf_from_pix_to_ous)


with open('sbgekinox_bcc-pixell_bfc.pkl', 'wb') as k:
    pickle.dump(tf_from_sbg_to_pix, k)


In [46]:
DI.print_transf_result(tf_from_sbg_to_pix)

translations :[-3.99027165 -0.03335964 -0.07475344], norm:3.9911112166154408
Ax Angle vector:[ 0.29681432 -0.06285438  0.95286441], angle (deg):0.8494067216957352
Euler Angle (deg):[ 0.25173069 -0.05516767  0.80924961]


In [51]:
#validation with Pixell:
result_tr_1 = np.load('/home/olivier/workspace/das/das/calib/examples/extrinsic_sbgekinox_ouster64/ckps9a/results.npy', allow_pickle=True).item()

In [52]:
M_trs = result_tr_1['calib']
len(M_trs)

500

In [53]:
M_trs[-1]

array([[ 0.99961686,  0.00714235,  0.02673987,  3.9239588 ],
       [-0.00727393,  0.9999619 ,  0.00482637, -0.02135073],
       [-0.02670437, -0.00501904,  0.99963075,  0.0628574 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32)

In [113]:
#optimization de la boucle 
# ouster -> sbg -> pixell -> ouster = 1 + epsilon

best_tf_from_sbg_to_ous = np.eye(4)
best_tf_from_sbg_to_pix = np.eye(4)
best_score = np.inf

list_tf_from_ous_to_sbg = np.load('/home/olivier/workspace/das/das/calib/examples/extrinsic_sbgekinox_ouster64/ckps7/results.npy', allow_pickle=True).item()['calib']

list_tf_from_pix_to_sbg = np.load('/home/olivier/workspace/das/das/calib/examples/extrinsic_sbgekinox_ouster64/ckps9a/results.npy', allow_pickle=True).item()['calib']

with open('/home/olivier/Leddartech/best_calibrations/pixset/extrinsics/icp_old/pixell_bfc-ouster64_bfc.pkl', 'rb') as k:
    tf_from_pix_to_ous = pickle.load(k)



In [114]:
tf_from_pix_to_ous

array([[ 0.99975172,  0.00156347, -0.02222718,  0.07268192],
       [-0.00161865,  0.99999565, -0.00246497, -0.00496503],
       [ 0.02222323,  0.00250034,  0.99974991, -0.2399816 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [115]:
len(list_tf_from_ous_to_sbg), len(list_tf_from_pix_to_sbg)

(50, 500)

In [116]:
for tf_ous_sbg in list_tf_from_ous_to_sbg:
    for tf_pix_sbg in list_tf_from_pix_to_sbg:
        delta_id = tf_ous_sbg @ linalg.tf_inv(tf_pix_sbg) @ tf_from_pix_to_ous - np.eye(4)
        _score = np.sum(delta_id**2)
        if _score < best_score:
            best_score = np.copy(_score)
            best_tf_from_sbg_to_ous = np.copy(linalg.tf_inv(tf_ous_sbg))
            best_tf_from_sbg_to_pix = np.copy(linalg.tf_inv(tf_pix_sbg))
            

In [117]:
best_score

array(7.30386275e-06)

In [106]:
best_tf_from_sbg_to_ous

array([[ 0.99965775, -0.00723096, -0.02514188, -3.9071717 ],
       [ 0.00710652,  0.9999621 , -0.00503542, -0.0111171 ],
       [ 0.02517733,  0.00485503,  0.9996712 , -0.3221957 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32)

In [107]:
linalg.tf_inv(best_tf_from_sbg_to_ous)

array([[ 0.99965775,  0.00710652,  0.02517733,  3.9140253 ],
       [-0.00723096,  0.9999621 ,  0.00485503, -0.01557165],
       [-0.02514188, -0.00503542,  0.9996712 ,  0.22380018],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32)

In [108]:
linalg.tf_inv(best_tf_from_sbg_to_pix)

array([[ 9.9993527e-01,  1.1282927e-02,  1.4698396e-03,  3.9903121e+00],
       [-1.1287323e-02,  9.9993175e-01,  3.0175715e-03, -2.2937423e-02],
       [-1.4356882e-03, -3.0339684e-03,  9.9999434e-01,  7.0996813e-02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32)

In [102]:
#saving_that
with open('best_v2_sbgekinox_bcc-ouster64_bfc.pkl', 'wb') as k:
    pickle.dump(best_tf_from_sbg_to_ous, k)

with open('best_v2_sbgekinox_bcc-pixell_bfc.pkl', 'wb') as k:
    pickle.dump(best_tf_from_sbg_to_pix, k)